In [1]:
import warnings
import pandas as pd
from tqdm import tqdm
import numpy as np
warnings.filterwarnings("ignore")

In [ ]:
raw_path = '..\data\geolife\geolife-all.csv'
tracks_data = pd.read_csv(raw_path, sep='\t')

split_ratio  = 0.8

all_poi = tracks_data['poi1'].drop_duplicates().values.tolist()
poi2idx = {poi:idx+1 for idx, poi in enumerate(all_poi)}
poi2idx['pad'] = 0


user_list = tracks_data['ObjectID'].drop_duplicates().values.tolist()
user_traj_dict = {key:[] for key in user_list}
for user_id in tqdm(tracks_data['ObjectID'].drop_duplicates().values.tolist()):
    one_user_data = tracks_data.loc[tracks_data.ObjectID == user_id, :]
    for traj_id in one_user_data['TrajNumber'].drop_duplicates().values.tolist():
        one_traj_data = one_user_data.loc[tracks_data.TrajNumber == traj_id, 'poi1'].values.tolist()
        user_traj_dict[user_id].append(one_traj_data)

user_traj_train, user_traj_test = {key:[] for key in user_list}, {key:[] for key in user_list}

for key in user_traj_dict:
    traj_num = len(user_traj_dict[key])

    for idx in list(range(traj_num))[:int(traj_num * split_ratio)]:
        user_traj_train[key].append([poi2idx[poi] for poi in user_traj_dict[key][idx]])
    
    for idx in list(range(traj_num))[int(traj_num * split_ratio):]:
        user_traj_test[key].append([poi2idx[poi] for poi in user_traj_dict[key][idx]])

print('poi number:', len(poi2idx))

In [3]:
def lcs(a, b):
    lena = len(a)
    lenb = len(b)
    c = [[0 for i in range(lenb + 1)] for j in range(lena + 1)]
    flag = [[0 for i in range(lenb + 1)] for j in range(lena + 1)]
    for i in range(lena):
        for j in range(lenb):
            if a[i] == b[j]:
                c[i + 1][j + 1] = c[i][j] + 1
                flag[i + 1][j + 1] = 'ok'
            elif c[i + 1][j] > c[i][j + 1]:
                c[i + 1][j + 1] = c[i + 1][j]
                flag[i + 1][j + 1] = 'left'
            else:
                c[i + 1][j + 1] = c[i][j + 1]
                flag[i + 1][j + 1] = 'up'
    calLcs.count = 0
    calLcs(flag, a, len(a), len(b))
    return calLcs.count / min(lena, lenb)


def calLcs(flag, a, i, j):
    if i == 0 or j == 0:
        return
    if flag[i][j] == 'ok':
        calLcs(flag, a, i - 1, j - 1)
        calLcs.count += 1
    elif flag[i][j] == 'left':
        calLcs(flag, a, i, j - 1)
    else:
        calLcs(flag, a, i - 1, j)

In [4]:
import heapq
from sklearn.metrics import f1_score, precision_score, recall_score

y_predict_list, y_true_list, acc1_list, acc5_list = [], [], [], []

for test_user in tqdm(user_traj_test):
    for one_test_traj in user_traj_test[test_user]:
        pred, target, sim = [], [], 0
        for train_user in user_traj_train:
            for one_train_traj in user_traj_train[train_user]:
                sim = lcs(one_test_traj, one_train_traj)
                pred.append(sim)
                target.append(train_user)
        predict_user = target[pred.index(max(pred))]
        y_predict_list.append(predict_user)
        y_true_list.append(test_user)
        acc1_list.append(predict_user == test_user)
        acc5_list.append(test_user in [target[idx] for idx in heapq.nlargest(5, range(len(pred)), pred.__getitem__)])

p = precision_score(y_true_list, y_predict_list, average='macro')
r = recall_score(y_true_list, y_predict_list, average='macro')
f1 = f1_score( y_true_list, y_predict_list, average='macro')

print("Result:\n acc1_test:{:.4f} \t  acc5_test:{:.4f} \t Macro-P:{:.4f} \t Macro-R:{:.4f} \t Macro-F1:{:.4f}".format(np.mean(acc1_list), np.mean(acc5_list), p, r ,f1))

100%|██████████| 90/90 [1:45:57<00:00, 70.64s/it]Result:
 acc1_test:0.3041 	  acc5_test:0.5800 	 Macro-P:0.3519 	 Macro-R:0.2657 	 Macro-F1:0.2634

